In [0]:
# Restarta python caso necessário.
from databricks.sdk.runtime import *
dbutils.library.restartPython()

# Import de Funções e Libs

As bibliotecas e funções utilizadas ao longo do código devem ser adicionadas aqui

In [0]:
import numpy as np
import pandas as pd
import requests
import json
import sherlock
import regras_politica as rp

### Testes Modulares - Referente ao tópico 2 - 2.1 Criação de flags, ranges, filtros

### Geração de Massa

In [0]:
#Gerando a massa modelo passando o caminho do JSON como nome da variável
base_entrada = pd.DataFrame({'solicitante.numeroDocumento':['123456789'],
                             'solicitante.dataAdmissaoFopag':["2002-11-05T17:18:43.888Z"],
                             'solicitante.dataRegistroPrimeiroContrato':["2002-11-05T17:18:43.888Z"],
                             'solicitante.dataRegistro':["2002-11-05T17:18:43.888Z"],
                             'solicitante.flagMAT30d':[0],
                             'solicitante.flagMAT60d':[0],
                             'solicitante.flagMAT90d':[0],
                             'solicitante.fonteRenda':['XYZ'],
                             'solicitante.flagPortabilidade':[0],
                             'solicitante.rendaLiquidaPicpay':[10000],
                             'solicitante.consumerID':[12345678987654321],
                             'solicitante.scoreAppSerasa':[999],
                             'solicitante.scoreInternoLongo':[999],
                             'solicitante.scoreInterno':[999],
                             'solicitante.investimentoTotal':[0.0],
                             'solicitante.flagInvestimentoPrincipalidade':[0],
                             'solicitante.flagPrincipalidadeAll':[0],
                             'solicitante.flagPrincipalidade':[0],
                             'solicitante.mesFlagPrincipalidade':[0],
                             'solicitante.flagAllowlist':["-99999"],
                             'solicitante.flagBlocklist':["-99999"],
                             'solicitante.scoreInternoCurto':[959],
                             'solicitante.scoreExterno':[959],
                             'solicitante.flagCartaoCreditoContratado':[0],
                             'solicitante.flagLimiteGarantidoContratado':[0]
                            })

In [0]:
#Criação das massas com casos forçados
#Digito CPF Concomitante CP
massa_DigitoCPFConcomitanteCP = sherlock.Gera_Massa(base_entrada)
massa_DigitoCPFConcomitanteCP.gera_cenario('solicitante.numeroDocumento',['123456789','123450089','123450089','123451189',
                                                               '123452289', '123453389','123454489','123455589',
                                                               '123456689','123457789','123458889','123459989',
                                                               '123452189','123452389','123453289','123457689',
                                                               '123450089','987654321','121212121'])
massa_DigitoCPFConcomitanteCP = massa_DigitoCPFConcomitanteCP.get_massa_final()

#RDG Grupo Small Limits
massa_RDGGrupoSmallLimits = sherlock.Gera_Massa(base_entrada)
massa_RDGGrupoSmallLimits.gera_cenario('solicitante.numeroDocumento',['123456789','123400789','123411789','123422789','123433789','123455789','123466789',
                                                                      '123477789','123488789','123499789','123445789','123454789','123432789','123423789',
                                                                      '121212121','987654321'])
massa_RDGGrupoSmallLimits = massa_RDGGrupoSmallLimits.get_massa_final()

##Data de admissão
massa_dataAdmissao = sherlock.Gera_Massa(base_entrada)
massa_dataAdmissao.gera_cenario('solicitante.dataAdmissaoFopag',["2002-11-05T17:18:43.888Z","2024-12-26T17:18:43.888Z","2024-01-20T00:00:00.888Z","1001-01-01"])
massa_dataAdmissao.gera_cenario('solicitante.dataRegistroPrimeiroContrato',["2002-11-05T17:18:43.888Z","2024-12-26T17:18:43.888Z","2024-01-20T00:00:00.888Z","1001-01-01"])
massa_dataAdmissao.gera_cenario('solicitante.dataRegistro',["2002-11-05T17:18:43.888Z","2024-12-26T17:18:43.888Z","2024-01-20T00:00:00.888Z","1001-01-01"])
massa_dataAdmissao = massa_dataAdmissao.get_massa_final()

#Data de admissão em meses
massa_dataAdmissaoMeses = sherlock.Gera_Massa(base_entrada)
massa_dataAdmissaoMeses.gera_cenario('solicitante.dataAdmissaoFopag',["2023-11-05T17:18:43.888Z","2024-02-26T10:00:00.888Z",
                                                                      "2024-01-26T10:00:00.888Z","2024-02-20T10:00:00.888Z",
                                                                      "1001-01-01"])
massa_dataAdmissaoMeses.gera_cenario('solicitante.dataRegistroPrimeiroContrato',["2023-11-05T17:18:43.888Z","2024-02-26T10:00:00.888Z",
                                                                                 "2024-01-26T10:00:00.888Z","2024-02-20T10:00:00.888Z",
                                                                                 "1001-01-01"])
massa_dataAdmissaoMeses.gera_cenario('solicitante.dataRegistro',["2023-11-05T17:18:43.888Z","2024-02-26T10:00:00.888Z",
                                                                 "2024-01-26T10:00:00.888Z","2024-02-20T10:00:00.888Z",
                                                                 "1001-01-01"])
massa_dataAdmissaoMeses = massa_dataAdmissaoMeses.get_massa_final()

#Cliente existente - 59, 60 e 61 dias de dif, aqui precisamos atualizar a data no momento de gerar a massa.
massa_clienteExistente = sherlock.Gera_Massa(base_entrada)
massa_clienteExistente.gera_cenario('solicitante.dataRegistro',["2024-02-09T16:48:17.760337Z",
                                                                "2024-02-08T16:48:17.760337Z",
                                                                "2024-02-07T16:48:17.760337Z",
                                                                "1001-01-01"])
massa_clienteExistente = massa_clienteExistente.get_massa_final()

#Cliente MAT
massa_clienteMAT = sherlock.Gera_Massa(base_entrada)
massa_clienteMAT.gera_cenario('solicitante.flagMAT30d',[0,1,2,3,4,5,6,7,8,9,-99999])
massa_clienteMAT = massa_clienteMAT.get_massa_final()

#ClienteMATRecorrencia
massa_clienteMATRecorrencia = sherlock.Gera_Massa(base_entrada)
massa_clienteMATRecorrencia.gera_cenario('solicitante.flagMAT30d',[0,1,2,3,4,5,6,7,8,9,-99999])
massa_clienteMATRecorrencia.gera_cenario('solicitante.flagMAT60d',[0,1,2,3,4,5,6,7,8,9,-99999])
massa_clienteMATRecorrencia.gera_cenario('solicitante.flagMAT90d',[0,1,2,3,4,5,6,7,8,9,-99999])
massa_clienteMATRecorrencia = massa_clienteMATRecorrencia.get_massa_final()

#Cliente Fopag
massa_clienteFoPag = sherlock.Gera_Massa(base_entrada)
massa_clienteFoPag.gera_cenario('solicitante.fonteRenda',['XYZ ABC','01. Fopag','01. FOPAG','1. Fopag','-1'])
massa_clienteFoPag.gera_cenario('solicitante.flagPortabilidade',[0,1,2,3,4,5,6,7,8,9,-99999])
massa_clienteFoPag = massa_clienteFoPag.get_massa_final()

#Renda inexistente
massa_rendaInexistente = sherlock.Gera_Massa(base_entrada)
massa_rendaInexistente.gera_cenario('solicitante.rendaLiquidaPicpay',[-99999,0,1,600,1000,10000])
massa_rendaInexistente = massa_rendaInexistente.get_massa_final()

#Cliente novo - 59, 60 e 61 dias de dif, aqui precisamos atualizar a data no momento de gerar a massa.
massa_clienteExistente = sherlock.Gera_Massa(base_entrada)
massa_clienteExistente.gera_cenario('solicitante.dataRegistro',["2024-02-09T16:48:17.760337Z",
                                                                "2024-02-08T16:48:17.760337Z",
                                                                "2024-02-07T16:48:17.760337Z",
                                                                "1001-01-01"])
massa_clienteExistente = massa_clienteExistente.get_massa_final()

#Nao Cliente (mar Aberto)
massa_naoCliente = sherlock.Gera_Massa(base_entrada)
massa_naoCliente.gera_cenario('solicitante.dataRegistro',["2024-01-20T13:27:50.863021Z",
                                                                "2024-01-19T13:27:50.863021Z",
                                                                "2024-01-18T13:27:50.863021Z",
                                                                "1001-01-01"])
massa_naoCliente.gera_cenario('solicitante.consumerID',[-99999,12345678987654321,999999999987654321])
massa_naoCliente = massa_naoCliente.get_massa_final()

#Score Serasa Inválido
massa_scoreSerasaInvalido = sherlock.Gera_Massa(base_entrada)
massa_scoreSerasaInvalido.gera_cenario('solicitante.scoreAppSerasa',[-99999,453,454,455,999])
massa_scoreSerasaInvalido = massa_scoreSerasaInvalido.get_massa_final()

#Sem Informação de Score - 59, 60 e 61 dias de dif, aqui precisamos atualizar a data no momento de gerar a massa.
massa_semInfoScore = sherlock.Gera_Massa(base_entrada)
massa_semInfoScore.gera_cenario('solicitante.consumerID',[-99999,12345678987654321])
massa_semInfoScore.gera_cenario('solicitante.scoreInternoLongo',[-99999,453,999])
massa_semInfoScore.gera_cenario('solicitante.scoreInterno',[-99999,453,999])
massa_semInfoScore.gera_cenario('solicitante.scoreExterno',[-99999,453,999])
massa_semInfoScore.gera_cenario('solicitante.dataRegistro',["2023-12-27T10:10:00.888Z",
                                                          "2023-12-28T10:10:00.888Z",
                                                          "2023-12-29T10:10:00.888Z",
                                                          "1001-01-01"])
massa_semInfoScore = massa_semInfoScore.get_massa_final()

#Valor Total Investido
massa_totalInvestido = sherlock.Gera_Massa(base_entrada)
massa_totalInvestido.gera_cenario('solicitante.investimentoTotal',[-99999,0,100,200,300,400,500,600,700,800,900,1000,2000,3000,4000,5000,6000])
massa_totalInvestido.gera_cenario('solicitante.investimentoTotal',[-99999,0,100,200,300,400,500,600,700,800,900,1000,2000,3000,4000,5000,6000])
massa_totalInvestido = massa_totalInvestido.get_massa_final()

#Percentual da Renda em investimento e #Investidor Original
massa_pctRendaInvest = sherlock.Gera_Massa(base_entrada)
massa_pctRendaInvest.gera_cenario('solicitante.investimentoTotal',[-99999,0,100,200,300,400,500,600,700,800,900,1000,2000,3000,4000,5000,6000])
massa_pctRendaInvest.gera_cenario('solicitante.rendaLiquidaPicpay',[-99999,0,200,300,301,1000])
massa_pctRendaInvest = massa_pctRendaInvest.get_massa_final()

#Principaliadde de investimento
massa_principalidadeInvestimento = sherlock.Gera_Massa(base_entrada)
massa_principalidadeInvestimento.gera_cenario('solicitante.investimentoTotal',[-99999,0,2000,3000])
massa_principalidadeInvestimento.gera_cenario('solicitante.rendaLiquidaPicpay',[-99999,0,1000])
massa_principalidadeInvestimento.gera_cenario('solicitante.flagInvestimentoPrincipalidade',[1,0])
massa_principalidadeInvestimento = massa_principalidadeInvestimento.get_massa_final()

#Principaliadde em todos os critérios
massa_principalidadeTodosCriterios = sherlock.Gera_Massa(base_entrada)
massa_principalidadeTodosCriterios.gera_cenario('solicitante.flagPrincipalidade',[1,0])
massa_principalidadeTodosCriterios.gera_cenario('solicitante.flagPrincipalidadeAll',[1,0])
massa_principalidadeTodosCriterios.gera_cenario('solicitante.mesFlagPrincipalidade',[-99999,0,1,2,3,4,5,6,7,8,9,10,11,12])
massa_principalidadeTodosCriterios = massa_principalidadeTodosCriterios.get_massa_final()

#Principalidade em todos os Critérios com Recorrência
massa_principalidadeRecorrencia = sherlock.Gera_Massa(base_entrada)
massa_principalidadeRecorrencia.gera_cenario('solicitante.flagPrincipalidade',[1,0])
massa_principalidadeRecorrencia.gera_cenario('solicitante.flagPrincipalidadeAll',[1,0])
massa_principalidadeRecorrencia.gera_cenario('solicitante.mesFlagPrincipalidade',[-99999,0,1,2,3,4,5,6,7,8,9,10,11,12])
massa_principalidadeRecorrencia = massa_principalidadeRecorrencia.get_massa_final()

#Cliente Blindado
massa_clienteBlindado = sherlock.Gera_Massa(base_entrada)
massa_clienteBlindado.gera_cenario('solicitante.flagAllowlist',["-99999","CaRtAo picpay CoNcEsSaO","cartao picpay concessao",
                                                                "Cartao PicPay Concessao","xpto Concessao","Cartao XYZ",
                                                                "Cartao PicPay ABC","Overall XYZ","Cartao Overall XYZ",
                                                                "xyz Overall","Overall","overall"
                                                                ])
massa_clienteBlindado = massa_clienteBlindado.get_massa_final()

#Cliente Bloqueado
massa_clienteBloqueado = sherlock.Gera_Massa(base_entrada)
massa_clienteBloqueado.gera_cenario('solicitante.flagBlocklist',["-99999","CaRtAo picpay CoNcEsSaO","cartao picpay concessao",
                                                                "Cartao PicPay Concessao","xpto Concessao","Cartao XYZ",
                                                                "Cartao PicPay ABC","Overall XYZ","Cartao Overall XYZ",
                                                                "xyz Overall","Overall","overall"
                                                                ])
massa_clienteBloqueado = massa_clienteBloqueado.get_massa_final()

#Faixa renda liquida
massa_faixaRendaLiquida = sherlock.Gera_Massa(base_entrada)
massa_faixaRendaLiquida.gera_cenario('solicitante.rendaLiquidaPicpay',[-99999,0,3999,4000,4001,5000,6000,7000,8000,9000,9999,10000,10001,99999])
massa_faixaRendaLiquida = massa_faixaRendaLiquida.get_massa_final()

#Faixa de Renda Bruta
massa_faixaRendaBruta = sherlock.Gera_Massa(base_entrada)
massa_faixaRendaBruta.gera_cenario('solicitante.rendaLiquidaPicpay',[-99999,0,3442.36,3442.37,3442.38,7499.15,7499.16,7499.17,11124.15,11124.16,11124.17])
massa_faixaRendaBruta = massa_faixaRendaBruta.get_massa_final()

#Faixa de Score Interno Curto
massa_faixaScoreInternoCurto = sherlock.Gera_Massa(base_entrada)
massa_faixaScoreInternoCurto.gera_cenario('solicitante.scoreInternoCurto',[-99999,0,731,732,733,806,807,808,846,847,848,
                                                                    882,883,884,899,900,901,913,914,915,925,
                                                                    926,927,937,938,939,953, 954,955,999])
massa_faixaScoreInternoCurto = massa_faixaScoreInternoCurto.get_massa_final()

#Faixa de Score Externo
massa_faixaScoreExterno = sherlock.Gera_Massa(base_entrada)
massa_faixaScoreExterno.gera_cenario('solicitante.scoreExterno',[-99999,0,879,880,881,907,908,909,922,923,924,934,935,936,943,944,945,951,952,953,958,959,960,965,966,967,970,971,972,999])
massa_faixaScoreExterno = massa_faixaScoreExterno.get_massa_final()

#Empilha todos casos com 1 caso padrão.
massa_final_flags_ranges_filtros = sherlock.Gera_Massa(base_entrada)
massa_final_flags_ranges_filtros.empilha_massas([massa_DigitoCPFConcomitanteCP,
                                                 massa_RDGGrupoSmallLimits,
                                                 massa_dataAdmissao,
                                                 massa_dataAdmissaoMeses,
                                                 massa_clienteExistente,
                                                 massa_clienteMAT,
                                                 massa_clienteMATRecorrencia,
                                                 massa_clienteFoPag,
                                                 massa_rendaInexistente,
                                                 massa_clienteExistente,
                                                 massa_naoCliente,
                                                 massa_scoreSerasaInvalido,
                                                 massa_semInfoScore,
                                                 massa_totalInvestido,
                                                 massa_pctRendaInvest,
                                                 massa_principalidadeInvestimento,
                                                 massa_principalidadeTodosCriterios,
                                                 massa_principalidadeRecorrencia,
                                                 massa_clienteBlindado,
                                                 massa_clienteBloqueado,
                                                 massa_faixaRendaLiquida,
                                                 massa_faixaRendaBruta,
                                                 massa_faixaScoreInternoCurto,
                                                 massa_faixaScoreExterno
])

massa_final_flags_ranges_filtros.set_bom_arqv()
massa_final_flags_ranges_filtros.converte_massa()

### Execução Modular e Geração de CSV

In [0]:
listaResultado = rp.execucaoModular(massa_final_flags_ranges_filtros.get_massa_json(),rp.CriacaoFlagsRangesFiltros)

In [0]:
import pandas as pd
results = []
for caso in listaResultado:
    results.append(caso['payloadHomol']['intermediarias']['clienteBloqueado'])
    
freqzinha = pd.DataFrame(results, columns=['valor'])
freqzinha['Freq'] = freqzinha['valor'].map(freqzinha['valor'].value_counts())
freqzinha = freqzinha.drop_duplicates().reset_index(drop=True)
freqzinha

In [0]:
# Lógica para validar, pode-se trocar o nome da variável na linha 6 para exibir uma freq dos dominios que essa variável possui.

import pandas as pd
results = []
for caso in listaResultado:
    results.append(caso['payloadHomol']['intermediarias']['clienteBlindado'])
    
freqzinha = pd.DataFrame(results, columns=['valor'])
freqzinha['Freq'] = freqzinha['valor'].map(freqzinha['valor'].value_counts())
freqzinha = freqzinha.drop_duplicates().reset_index(drop=True)
freqzinha

In [0]:
# Renomeação das colunas para o cabeçalho do RMA
massa_final_flags_ranges_filtros.rename_cabecalho()

In [0]:
massa_final_flags_ranges_filtros.atribui_expected(listaResultado,'payloadHomol.intermediarias.digitoCPFConcomitanteCP','cpf6e7Digito')
massa_final_flags_ranges_filtros.atribui_expected(listaResultado,'payloadHomol.intermediarias.cpf5e6Digito','cpf5e6Digito')
massa_final_flags_ranges_filtros.atribui_expected(listaResultado,'payloadHomol.intermediarias.dataAdmissao','dataAdmissao')
massa_final_flags_ranges_filtros.atribui_expected(listaResultado,'payloadHomol.intermediarias.admissaoMeses','dataAdmissaoMeses')
massa_final_flags_ranges_filtros.atribui_expected(listaResultado,'payloadHomol.intermediarias.flagClienteExistente','clienteExistente')
massa_final_flags_ranges_filtros.atribui_expected(listaResultado,'payloadHomol.intermediarias.clienteMAT','clienteMAT')
massa_final_flags_ranges_filtros.atribui_expected(listaResultado,'payloadHomol.intermediarias.MATRecorrente','clienteMATRecorrencia')
massa_final_flags_ranges_filtros.atribui_expected(listaResultado,'payloadHomol.intermediarias.clienteFoPag','clienteFopag')
massa_final_flags_ranges_filtros.atribui_expected(listaResultado,'payloadHomol.intermediarias.rendaInexistente','rendaInexistente')
massa_final_flags_ranges_filtros.atribui_expected(listaResultado,'payloadHomol.intermediarias.flagClienteNovo','clienteNovo')
massa_final_flags_ranges_filtros.atribui_expected(listaResultado,'payloadHomol.intermediarias.flagNaoCliente' ,'naoCliente')
massa_final_flags_ranges_filtros.atribui_expected(listaResultado,'payloadHomol.intermediarias.serasaInvalido','scoreSerasaInvalido')
massa_final_flags_ranges_filtros.atribui_expected(listaResultado,'payloadHomol.intermediarias.semInfoScore','scoreInexistente')
massa_final_flags_ranges_filtros.atribui_expected(listaResultado,'payloadHomol.intermediarias.valorTotalInvestido','vlrTotalInvestido')
massa_final_flags_ranges_filtros.atribui_expected(listaResultado,'payloadHomol.intermediarias.percentRendaInvestimento','percentRendaInvestimento')
massa_final_flags_ranges_filtros.atribui_expected(listaResultado,'payloadHomol.intermediarias.investidorOriginal','investidorOriginal')
massa_final_flags_ranges_filtros.atribui_expected(listaResultado,'payloadHomol.intermediarias.principalidadeDeInvestimento','principalidadeInvestimento')
massa_final_flags_ranges_filtros.atribui_expected(listaResultado,'payloadHomol.intermediarias.principalidadeAll','principalidadeAll')
massa_final_flags_ranges_filtros.atribui_expected(listaResultado,'payloadHomol.intermediarias.recorrenciaPrincipalidade','principalidadeAllRecorrencia')
massa_final_flags_ranges_filtros.atribui_expected(listaResultado,'payloadHomol.intermediarias.clienteBlindado','clienteBlindado')
massa_final_flags_ranges_filtros.atribui_expected(listaResultado,'payloadHomol.intermediarias.clienteBloqueado','clienteBloqueado')
massa_final_flags_ranges_filtros.atribui_expected(listaResultado,'payloadHomol.intermediarias.faixaRendaLiquida','faixaRendaLiquida')
massa_final_flags_ranges_filtros.atribui_expected(listaResultado,'payloadHomol.intermediarias.faixaRendaBruta','faixaRendaBruta')
massa_final_flags_ranges_filtros.atribui_expected(listaResultado,'payloadHomol.intermediarias.faixaScoreInternoCurto','faixaScoreInternoCurto')
massa_final_flags_ranges_filtros.atribui_expected(listaResultado,'payloadHomol.intermediarias.faixaScoreExterno','faixaScoreExterno')
massa_final_flags_ranges_filtros.get_massa_dm()

### Conversão da massa para CSV

In [0]:
massa_final_flags_ranges_filtros.get_massa_dm().to_csv(r'/Workspace/Users/kaue.ongaro@picpaybank.com/2 - Homologação/Concessao de Cartoes - Online/massas/massaFlagsRangesFiltros.csv',index=False)

#### Análises

In [0]:
dataID = 228
print(json.dumps(listaResultado[dataID-1],indent=4))